In [ ]:
import numpy as np
import random
#Idea:
#Scanners have a list of beacons, and those beacons have pairwise distances
#We can posit scanners are in overlapping areas if there are at least C(12,2) matching distances
#We can add a verify for beacons not in overlap once scanner positions are resolved
#Given two scanners in overlapping areas, we can take a random set of beacons with matching distance profiles across scanners.
# If b0 is beacon in scanner 0 coord and b1 is beacon in scanner 1 coord
#b1 =  A b0  + C, for some A and C, extend the vector [b0;1]  and solve for [b1_1, b1_2, b1_3, b1_4] = X [b1_1, b1_2, b1_3, b1_4]
#Where X is [A|C]
#Alternatively, can do, using rows and flipping around, b0 =  b1 R + S
#We can tranform b1 coords to b0 coord and replace. Now scanner 1 is oriented like scanner 0 and coordinates are shifted to match scanner 0
#Can run verify checks here to ensure the overlap is consistent.

#Start with a seed scanner, scanner 0, and put in the orientated scanner list.
#While the orientated scanner list is not empty, pop a scanner off it, and put all it's oreinted beacons in a dict.
#Go through each scanner in the unoriented list and find any scanners that pair with it using the algorithm above, 
#Remove those scanners from the unoriented list, add them to the oriented list.

class Scanner:
    def __init__(self):
        self.beacons = []
        self.beaconDistances = []
        self.pairwiseBeaconDistances = []
    def __lt__(self, other):
        if self.cost < other.cost:
            return True
        return False
    def applyTransformToBeacons(self, X):
        for i in range(0,len(self.beacons)):
            b = np.concatenate((self.beacons[i], [1]))
            self.beacons[i] = np.dot(b, X)
        return

    

def readInput(input):
    #Grab the scanners
    scanners = []
    lines = input.split("\n")
    currScanner = None
    for line in lines:
        if line == "":
            continue
        #New Scanner
        if "---" in line:
            if currScanner:
                scanners.append(currScanner)
            currScanner = Scanner()
            continue
        vals = line.split(",")
        beacon = [int(vals[0]), int(vals[1]), int(vals[2])]
        currScanner.beacons.append(np.array(beacon))    
    scanners.append(currScanner)
    
    
    #Calculate its beacon fingerprint, e.g. the distances and sort for convenience
    for scanner in scanners:
        for i in range(0, len(scanner.beacons)):
            for j in range(i+1, len(scanner.beacons)):
                A, B = scanner.beacons[i], scanner.beacons[j]
                scanner.beaconDistances.append(abs(A[0]-B[0]) + abs(A[1]-B[1])+ abs(A[2]-B[2]))
        scanner.beaconDistances.sort()
        
        #Pre-calculate the all beacon distances
        scanner.pairwiseBeaconDistances = getPairwiseBeaconDistances(scanner)
    return scanners


#Get the pairwise beacon distances for scanner A
def getPairwiseBeaconDistances(A):
    distancesA = []
    for i in range(0, len(A.beacons)):
        currBeacon = []
        for j in range(0, len(A.beacons)):
            if i==j:
                continue
            X, Y = A.beacons[i], A.beacons[j]
            d = abs(X[0]-Y[0])+abs(X[1]-Y[1])+abs(X[2]-Y[2])
            currBeacon.append(d)
        currBeacon.sort()
        distancesA.append(currBeacon)
    return distancesA


#This function takes two SORTED lists and returns the number of matching entries
def countMatchingEntries(A,B):
    r = A.copy()
    s = B.copy()
    
    cnt = 0
    while len(r)>0 and len(s)>0:
        currR, currS = r[-1], s[-1]
        if currR == currS:
            cnt += 1
            r.pop()
            s.pop()
            continue
        if currR < currS:
            s.pop()
            continue
        r.pop()
    return cnt


#Returns true if A and B have 12 scanners that match (C(12,2) distances)
#beaconDistances are already sorted in ascending order
#we treat this as a stack where we compare the top elements
def hasMatchingFingerprint(A, B, threshold = (12*11)/2):
    cnt = countMatchingEntries(A.beaconDistances, B.beaconDistances)
    if cnt >= threshold:
        return True
    return False

#Returns the two subsets of beacons that match in order
#This algorithm does a brute-force to determine if the beacons can match.
def findMatchingBeacons(A,B, threshold = 11):
    matchingBeaconsA = []
    matchingBeaconsB = []
    beaconDistancesA = A.pairwiseBeaconDistances
    beaconDistancesB = B.pairwiseBeaconDistances
    for i in range(0,len(beaconDistancesA)):
        for j in range(0, len(beaconDistancesB)):
            cnt = countMatchingEntries(beaconDistancesA[i], beaconDistancesB[j])
            if cnt >= threshold:
                matchingBeaconsA.append(i), matchingBeaconsB.append(j)
                break
    return matchingBeaconsA, matchingBeaconsB
    
def getFourUnique(N):
    vals = []
    if N <= 4:
        retrun [1,2,3,4]    
    while len(vals)<4:
        x = random.randint(0, N-1)        
        if x in vals:
            continue
        vals.append(x)
    return vals
#Get Matrix for transform b0 = R b1 + S
def calculateTransform(scannerA, matchingBeaconsA, scannerB, matchingBeaconsB, numTries = 0):
    if numTries >= 10:
        print("Too many failed attempts")
        return None
    B0 = np.zeros((4,4))
    B1 = np.zeros((4,4))
    N = len(matchingBeaconsA)
    indices = getFourUnique(N)
    for i in indices:
        B0[:,i] = scannerA.beacons[matchingBeaconsA[i]]
        B1[:,i] = scannerB.beacons[matchingBeaconsB[i]]
    #Solve for X, B0 = X B1
    return T
    
def checkTransform(scannerA, matchingBeaconsA, scannerB, matchingBeaconsB, T, threshold = 12):
    numFit = 0
    for i in range(0, len(matchingBeaconsA)):
        B0 = scannerA.beacons[matchingBeaconsA[i]]
        B1 = scannerB.beacons[matchingBeaconsB[i]]
        augmented = np.concatenate((B1,[1]))
        if np.allclose(np.dot(augmented, T), B0):
            numFit+=1
    if numFit >=threshold:
        return True
    return False



#First element of the transform determines the coordinate system
#R and S are the matching beacon indices as returned by find matching beacons
def getTransform(scanner0, scanner1, R, S):
    if len(R) <4 or len(S) <4:
        return None
    
    B0 = np.zeros((4,3)) #4 beacons of size 3
    B1 = np.zeros((4,4)) #4 beacons of size 3 with an appended 1
    N = len(R)
    indices = getFourUnique(N)
    for cnt in range(0, len(indices)):
        i = indices[cnt]
        u = scanner0.beacons[R[i]]
        vpart = scanner1.beacons[S[i]]
        v = np.concatenate((vpart, [1]))
        B0[cnt,:] = u
        B1[cnt,:] = v
    X = np.linalg.solve(B1, B0)
    #print(np.allclose(np.dot(B1, X), B0))
    return np.rint(X)

def addBeaconsToDict(beaconDict, currScanner):
    for ba in currScanner.beacons:
        beaconDict[ba.__str__().replace(".", "")] = True
    return
        
def matchAllScanners(scanners):
    beaconDict = {}
    root = scanners.pop(0)
    orientedScanners = [root]
    
    while len(orientedScanners)> 0:
        currScanner = orientedScanners.pop()
        addBeaconsToDict(beaconDict, currScanner)
        print("Beacon Dict now has size: ", len(beaconDict))
        
        for sindex in range(len(scanners)-1,-1, -1):
            testScanner = scanners[sindex]
            if hasMatchingFingerprint(currScanner,testScanner):
                B0,B1 = findMatchingBeacons(currScanner,testScanner, threshold = 11)
                X = getTransform(currScanner, testScanner, B0, B1)
                if X.any == None:
                    print("Error, False positive fingerprint, could not construct the transform")
                    continue
                if not checkTransform(currScanner, B0, testScanner, B1, X):
                    print("Error, False positive fingerprint, transform did not check")
                    continue
                #We have a match, remove, orient and append
                testScanner.applyTransformToBeacons(X)
                orientedScanners.append(scanners.pop(sindex))
    if len(scanners) != 0:
        print("Error, could not identify {} scanners".format(len(scanners)))

scanners = readInput(input)
matchAllScanners(scanners)





In [ ]:
def getScannerLocation(scanner, X):
    b = np.array([0,0,0,1])
    return np.dot(b,X)

def reportMaxScannerDifferences(scanners):
    beaconDict = {}
    root = scanners.pop(0)
    orientedScanners = [root]
    locations = [np.array([0,0,0])]
    
    while len(orientedScanners)> 0:
        currScanner = orientedScanners.pop()
        addBeaconsToDict(beaconDict, currScanner)
        print("Beacon Dict now has size: ", len(beaconDict))
        
        for sindex in range(len(scanners)-1,-1, -1):
            testScanner = scanners[sindex]
            if hasMatchingFingerprint(currScanner,testScanner):
                B0,B1 = findMatchingBeacons(currScanner,testScanner, threshold = 11)
                X = getTransform(currScanner, testScanner, B0, B1)
                if X.any == None:
                    print("Error, False positive fingerprint, could not construct the transform")
                    continue
                if not checkTransform(currScanner, B0, testScanner, B1, X):
                    print("Error, False positive fingerprint, transform did not check")
                    continue
                #We have a match, remove, orient, aappend, and store location
                testScanner.applyTransformToBeacons(X)
                locations.append(getScannerLocation(testScanner, X))
                orientedScanners.append(scanners.pop(sindex))
    if len(scanners) != 0:
        print("Error, could not identify {} scanners".format(len(scanners)))
        
    #Now that we have all the locations find the max distance scanner pairs
    maxDistance = 0
    for i in range(0, len(locations)):
        for j in range(i+1,len(locations)):
            x = locations[i]
            y = locations[j]
            distance = abs(x[0]-y[0])+abs(x[1]-y[1])+abs(x[2]-y[2])
            if distance > maxDistance:
                maxDistance = distance
    return maxDistance

scanners = readInput(input)
maxDistance = reportMaxScannerDifferences(scanners)
print("Max distance between scanners is: ", maxDistance)

In [ ]:
input = """--- scanner 0 ---
0,2,0
4,1,0
3,3,0

--- scanner 1 ---
-1,-1,0
-5,0,0
-2,1,0"""

In [ ]:
input = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390"""

In [ ]:
input = """--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14"""

In [ ]:
#Test code:


#A = scanners[0]
#B = scanners[1]
#print(scanners[0].beaconDistances)
#print(scanners[1].beaconDistances)
#print(scanners[0].pairwiseBeaconDistances)
#print(scanners[1].pairwiseBeaconDistances)
#print(countMatchingEntries([1,2,3,4,5,6], [2,4,5,8,9,11]), 3)
#print(countMatchingEntries(scanners[0].beaconDistances,scanners[1].beaconDistances), 12*11/2)
#print(hasMatchingFingerprint(scanners[0],scanners[1]))
#R,S = findMatchingBeacons(A,B, threshold = 11)
#X = getTransform(A, B, R, S)
#print(checkTransform(A, R, B, S, X))
#B.applyTransformToBeacons(X)
#beaconDict = {}
#for bb in B.beacons:
#    beaconDict[bb.__str__().replace(".", "")] = True

#print(len(A.beacons), len(B.beacons), len(beaconDict))
 